In [ ]:
import torch
import os
import numpy as np
import pandas as pd

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [2]:
import resource
import logging
resource.setrlimit(resource.RLIMIT_CORE, (0, 0))
logging.info("Coredump generation has been disabled.")

In [ ]:
# Function to Display Available GPUs
def print_available_gpus():
    num_gpus = torch.cuda.device_count()
    print(f"Number of available GPUs: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

print_available_gpus()

Number of available GPUs: 2
GPU 0: NVIDIA RTX A6000
GPU 1: NVIDIA RTX A6000


In [4]:
import os
#os.chdir('/home/code/import_file')
os.chdir('/home/code/features')

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from torch.cuda.amp import GradScaler, autocast  # For Mixed Precision Training

import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
from datetime import datetime

import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import time

import logging
from datetime import datetime
from torchcrf import CRF

In [ ]:
import subprocess

# ===== User-configurable parameters =====
mode = 'span'  # 'pooling' or 'span'
span_decoder = 'crf'  # 'crf' or 'linear'
feature_type = 'norm'  # 'raw' or 'norm'
features_to_use = "key_avg"  # Select features from key_avg, query_entropy, key_entropy, and lookback_ratio.
                             # To select multiple features, separate them with commas, e.g., "key_avg,query_entropy"

layers = "0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31"  # Layers to use
heads = "0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31"  # Heads to use

clf_mode = 'transformer'  # 'transformer' or 'lookback_ratio_lr' # Implementation of a linear regression model for lookback_ratio

model_type = 'llama'  # 'llama' or 'qwen'
dataset_name = 'summary'  # 'qa', 'data2txt', 'summary', 'truthqa'

absolute_path = f'/home/code/data/saves'  # Path to feature data
train_path = f'{absolute_path}/{model_type}_{dataset_name}_train.pkl'
val_path = f'{absolute_path}/{model_type}_{dataset_name}_val.pkl'
test_path = f'{absolute_path}/{model_type}_{dataset_name}_test.pkl'

if features_to_use == 'key_avg':
    features_to_use_name = 'one'

elif features_to_use == 'key_avg,query_entropy,key_entropy':
    features_to_use_name = 'all'

else:
    features_to_use_name = 'others'

if mode == 'span':
    save_name = f'{model_type}_{dataset_name}_{features_to_use_name}'

    if span_decoder == 'crf':
        save_name += '_crf'
    
    if clf_mode == 'lookback_ratio_lr':
        save_name += '_lookback_ratio_lr'

elif mode == 'pooling':
    pooling_type = 'mean'  # Choose one from mean, max, cls, attention, or count
    save_name = f'{model_type}_{dataset_name}_{features_to_use_name}_{pooling_type}'

script_path_optimize = '/home/code/features/optimize.py'  # Training script
script_path_evaluate = '/home/code/features/evaluate.py'  # Evaluation script

n_epochs = 150
n_trials = 200
batch_size = 64

In [ ]:
# Training script execution command
command_optimize = [
    'python', script_path_optimize,
    '--mode', mode,
    '--dataset_name', dataset_name,
    '--feature_type', feature_type,
    '--save_name', save_name,
    '--train_path', train_path,
    '--val_path', val_path,
    '--test_path', test_path,
    '--n_epochs', str(n_epochs),
    '--n_trials', str(n_trials),
    '--batch_size', str(batch_size),
    '--features_to_use', features_to_use,
    '--layers_to_use', layers,
    '--heads_to_use', heads,
    '--clf_mode', clf_mode,
    '--span_decoder', span_decoder,
    '--model_type', model_type
]

if mode == 'pooling':  # Specify only when using pooling mode
    command_optimize += ['--pooling_type', pooling_type]

# `top_models_json_path` is used when applying parameters from another model.
# If not used, comment it out.
#top_models_json_path = '/home/code/data/training_file/llama_data2txt_all_norm/top5_transformer_models_llama_data2txt_all_norm.json'
#if top_models_json_path:
    #command_optimize += ['--top_models_json_path', top_models_json_path]

try:
    result = subprocess.run(command_optimize, capture_output=True, text=True, check=True)
    print("=== Training Script Execution Result ===")
    print("Return code:", result.returncode)
    print("Standard Output:")
    print(result.stdout)
    print("Standard Error:")
    print(result.stderr)
except subprocess.CalledProcessError as e:
    print("An error occurred while running the training script.")
    print("Return code:", e.returncode)
    print("Standard Output:")
    print(e.stdout)
    print("Standard Error:")
    print(e.stderr)
except FileNotFoundError as e:
    print("Training script file not found. Please check the script path.")
    print(e)
except Exception as e:
    print("An unexpected error occurred during training.")
    print(e)

  0%|          | 0/2 [00:00<?, ?it/s]

=== 学習スクリプト実行結果 ===
Return code: 0
Standard Output:
----- トップモデルの学習開始: params={'model_dim': 256, 'num_heads': 16, 'num_layers': 6, 'dropout_prob': 0.48151508470374516, 'lr': 0.00036040810688745304, 'weight_decay': 0.0001271769521844205} -----
トップモデルを /home/code/data1/training_file/20250214_qwen_norm_summary_attention_only_norm/models/top_model_qwen_ragtruth_summary_20250214_qwen_norm_summary_attention_only_norm_model_dim_256_layers_6_heads_16_rank_1.pt に保存しました。
評価指標を /home/code/data1/training_file/20250214_qwen_norm_summary_attention_only_norm/models/eval_metrics_model_dim_256_layers_6_heads_16_rank_1.json に保存しました。
----- トップモデルの学習開始: params={'model_dim': 256, 'num_heads': 32, 'num_layers': 6, 'dropout_prob': 0.498347641955638, 'lr': 0.0004035303242822194, 'weight_decay': 0.00010164691556857959} -----
----- トップモデルの学習開始: params={'model_dim': 256, 'num_heads': 32, 'num_layers': 6, 'dropout_prob': 0.47439365866985805, 'lr': 0.00032191371031902724, 'weight_decay': 9.620883997028516e-05} ---

KeyboardInterrupt: 

In [ ]:
# Evaluation script execution command
command_evaluate = [
    'python', script_path_evaluate,
    '--mode', mode,
    '--dataset_name', dataset_name,
    '--feature_type', feature_type,
    '--save_name', save_name,
    '--train_path', train_path,
    '--val_path', val_path,
    '--test_path', test_path,
    '--batch_size', str(batch_size),
    '--features_to_use', features_to_use,
    '--layers_to_use', layers,
    '--heads_to_use', heads,
    '--clf_mode', clf_mode,
    '--span_decoder', span_decoder,
    '--model_type', model_type
]

if mode == 'pooling':  # Specify only when using pooling mode
    command_evaluate += ['--pooling_type', pooling_type]

try:
    result = subprocess.run(command_evaluate, capture_output=True, text=True, check=True)
    print("=== Evaluation Script Execution Result ===")
    print("Return code:", result.returncode)
    print("Standard Output:")
    print(result.stdout)
    print("Standard Error:")
    print(result.stderr)
except subprocess.CalledProcessError as e:
    print("An error occurred while running the evaluation script.")
    print("Return code:", e.returncode)
    print("Standard Output:")
    print(e.stdout)
    print("Standard Error:")
    print(e.stderr)
except FileNotFoundError as e:
    print("Evaluation script file not found. Please check the script path.")
    print(e)
except Exception as e:
    print("An unexpected error occurred during evaluation.")
    print(e)